In [7]:
def mat_multiply(A,B):
    rows_A,cols_A = len(A),len(A[0])
    rows_B,cols_B = len(B),len(B[0])
    
    if cols_A != rows_B:
        print("矩阵A与矩阵B不能相乘")
        
    C = [[0 for _ in range(cols_B)]for _ in range(rows_A)]
    
    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                C[i][j] += A[i][k] * B[k][j]
    
    return C
def mat_multipy(A,B):
    rows_A,cols_A =len(A),len(A[0])
    C=[[s for _ in range(cols_B)] for _ in range(rows_A)]

In [10]:
%%time
import numpy as np
A=np.array([[1,2,3],
           [4,5,6]])
B=np.array([[1,2],
           [3,4],
           [5,6]])
result = mat_multiply(A,B)
print(A)
print(B)
print(result)

[[1 2 3]
 [4 5 6]]
[[1 2]
 [3 4]
 [5 6]]
[[np.int64(22), np.int64(28)], [np.int64(49), np.int64(64)]]
CPU times: user 362 μs, sys: 0 ns, total: 362 μs
Wall time: 369 μs


In [ ]:
%%time
result = np.matmul(A,B)
print(result)

In [ ]:
from numba import cuda
import numpy as np

@cuda.jit
def mat_multiply_cuda_kernel(A, B, C):
    # 获取线程的全局索引
    row, col = cuda.grid(2)
    
    # 检查是否超出矩阵范围
    if row < C.shape[0] and col < C.shape[1]:
        temp = 0
        for k in range(A.shape[1]):
            temp += A[row, k] * B[k, col]
        C[row, col] = temp

def mat_multiply_cuda_numba(A, B):
    # 检查矩阵是否可以相乘
    if A.shape[1] != B.shape[0]:
        raise ValueError("矩阵A与矩阵B不能相乘")
    
    # 初始化结果矩阵
    C = np.zeros((A.shape[0], B.shape[1]), dtype=A.dtype)
    
    # 定义线程和块的大小
    threads_per_block = (16, 16)
    blocks_per_grid = ((C.shape[0] + threads_per_block[0] - 1) // threads_per_block[0],
                       (C.shape[1] + threads_per_block[1] - 1) // threads_per_block[1])
    
    # 将矩阵复制到GPU
    A_gpu = cuda.to_device(A)
    B_gpu = cuda.to_device(B)
    C_gpu = cuda.to_device(C)
    
    # 启动CUDA内核
    mat_multiply_cuda_kernel[blocks_per_grid, threads_per_block](A_gpu, B_gpu, C_gpu)
    
    # 将结果复制回主机
    C = C_gpu.copy_to_host()
    
    return C

In [ ]:
result = mat_multiply_cuda_numba(A,B)